**NOTE:** The below implementation is highly inspired from he PINN paper from 2019 and their open source repository, the data and model architecture has been referenced from there.

In [ ]:
# importing the libraries

import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
from pyDOE import lhs

### Data Preparation

In [3]:
# Domain boundaries

x_min, x_max = -5.0, 5.0
t_min, t_max = 0.0, np.pi/2

lb = np.array([x_min, t_min])
ub = np.array([x_max, t_max])

# lb - lower bound, ub - upper bound

N0 = 20
N_b = 20
N_f = 10000

# N0 - number of points in the initial condition, 
# N_b - number of points in the boundary condition, 
# N_f - number of points in the interior domain

layers = [2, 50, 50, 50, 50, 2]

In [6]:
data = scipy.io.loadmat('../data/NLS.mat')

In [11]:
list(data.keys())

['__header__', '__version__', '__globals__', 'tt', 'uu', 'x']

`tt` - Time Grid

`uu` - Full Solution Array

`x` - Spatial Grid

In [14]:
print(f"Shape of time grid: {data['tt'].shape}")
print(f"Shape of solution array: {data['uu'].shape}")
print(f"Shape of spatial grid: {data['x'].shape}")


Shape of time grid: (1, 201)
Shape of solution array: (256, 201)
Shape of spatial grid: (1, 256)


In [18]:
t = data['tt'].flatten()[:, None]
x = data['x'].flatten()[:, None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)

Exact_h_modulus = np.sqrt(Exact_u**2 + Exact_v**2)

# h = u + iv
# h_modulus = sqrt(u^2 + v^2)

In [17]:
print(f"Shape of exact solution: {Exact.shape}")
print(f"Shape of time grid: {t.shape}")
print(f"Shape of spatial grid: {x.shape}")


Shape of exact solution: (256, 201)
Shape of time grid: (201, 1)
Shape of spatial grid: (256, 1)


In [20]:
X, T = np.meshgrid(x, t)

X_star = np.hstack((X.flatten()[:, None], T.flatten()[:, None]))

u_star = Exact_u.flatten()[:, None]
v_star = Exact_v.flatten()[:, None]
h_star = Exact_h_modulus.flatten()[:, None]

print(f"Shape of u_star: {u_star.shape}")
print(f"Shape of v_star: {v_star.shape}")
print(f"Shape of h_star: {h_star.shape}")
print(f"Shape of X_star: {X_star.shape}")

# X_star - spatial and temporal grid
# u_star - real part of the solution
# v_star - imaginary part of the solution
# h_star - modulus of the solution

Shape of u_star: (51456, 1)
Shape of v_star: (51456, 1)
Shape of h_star: (51456, 1)
Shape of X_star: (51456, 2)


In [23]:
idx_x = np.random.choice(x.shape[0], N0, replace=False)

x0 = x[idx_x,:]
u0 = Exact_u[idx_x,0:1]
v0 = Exact_v[idx_x,0:1]

idx_t = np.random.choice(t.shape[0], N_b, replace=False)
t_b = t[idx_t,:]

X_f = lb + (ub - lb) * lhs(2, N_f)

# x0 - selected x points from the initial condition where t = 0
# u0 - real part of the initial condition
# v0 - imaginary part of the initial condition


# t_b - time values at which the spatial boundaries will be sampled.
# X_f - interior points (xf, tf) used to enforce the PDE via the residuals f_u and f_v.


`lhs(2, N_f)`: Latin Hypercube Sampling of 2D space ([0,1]^2) to get N_f points.

| Set                       | How sampled                     | Domain             | Purpose                           |
| ------------------------- | ------------------------------- | ------------------ | --------------------------------- |
| `x0, u0, v0`              | random `N0` spatial points      | t=0                | **initial condition**             |
| `tb` (used in X_lb, X_ub) | random `Nb` time points         | x=lb, ub           | **boundary condition**            |
| `X_f`                     | `Nf` points via Latin Hypercube | interior 2D domain | **collocation / PDE enforcement** |


Now that the data preparation is complete, we move on to **modeling the neural network**. 

- The datasets `x0`, `u0`, `v0`, `tb`, and `X_f` are used to **compute the PINN’s loss function** during training:

  - **Initial condition points** (`x0`, `u0`, `v0`):  
    These are spatial points at \(t=0\) and enforce the initial state of the system.

  - **Boundary condition points** (`tb`, `lb`, `ub`):  
    These correspond to time points at the spatial boundaries and enforce constraints at the edges of the domain.

  - **Collocation points** (`X_f`):  
    These are points in the interior of the domain where the PDE residuals are enforced, ensuring the neural network satisfies the governing equations.

- The `_star` datasets (e.g., `X_star`, `u_star`, `v_star`, `h_star`) represent the **full reference solution on a dense grid**, which is used for **prediction, evaluation, and error computation** after training. These points are **not used in computing the loss**, but serve as a benchmark to assess the accuracy of the network’s predictions.

**Summary:**  
- `x0, u0, v0, tb, X_f` → **training points** used to enforce physics, initial, and boundary conditions.  
- `_star` values → **evaluation points** used to validate the model’s performance.

In [ ]:
# creating the PINN model

class PhysicsInformedNN:
    # Initialize the PINN model
    def __init__(self, x0, t0, u0, v0, tb, lb, ub, X_f, layers, lr):
        """
        Initialize the PhysicsInformedNN model.

        Args:
            x0 (np.ndarray): Initial spatial points
            t0 (np.ndarray): Initial time points
            u0 (np.ndarray): Initial real part of solution
            v0 (np.ndarray): Initial imaginary part of solution
            tb (np.ndarray): Boundary time points
            lb (np.ndarray): Lower spatial boundary
            ub (np.ndarray): Upper spatial boundary
            X_f (np.ndarray): Collocation points
            layers (list): List of layer sizes for the neural network
            lr (float): Learning rate for the optimizer
        """      
        self.layers = layers
        self.lr = lr
        self.weights, self.biases = self.initialize_NN(layers)
        
        # what are we predicting?
        # u,v at initial condition (u0, v0)
        # u,v at boundary (u_lb, v_lb, u_ub, v_ub)
        # f_u, f_v at interior points (f_u_int, f_v_int)
        
        # constructing input data for the model for all the above conditions

        X0 = np.concatenate((x0, np.zeros_like(x0)), 1) # (x0, t=0)
        X_lb = np.concatenate((lb[0]*np.ones_like(tb), tb), 1) # (x=lb, t)
        X_ub = np.concatenate((ub[0]*np.ones_like(tb), tb), 1) # (x=ub, t)
        
        self.lb = tf.constant(lb, dtype=tf.float32)
        self.ub = tf.constant(ub, dtype=tf.float32)
        self.u0 = tf.constant(u0, dtype=tf.float32)
        self.v0 = tf.constant(v0, dtype=tf.float32)
        
        self.x0 = tf.constant(X0[:,0:1], dtype=tf.float32)
        self.t0 = tf.constant(X0[:,1:2], dtype=tf.float32)
        self.x_lb = tf.constant(X_lb[:,0:1], dtype=tf.float32)
        self.t_lb = tf.constant(X_lb[:,1:2], dtype=tf.float32)
        self.x_ub = tf.constant(X_ub[:,0:1], dtype=tf.float32)
        self.t_ub = tf.constant(X_ub[:,1:2], dtype=tf.float32)

        self.x_f = tf.constant(X_f[:,0:1], dtype=tf.float32)
        self.t_f = tf.constant(X_f[:,1:2], dtype=tf.float32)
        
    def build_nn(self):
        """Forward pass through the neural network."""
        model = tf.keras.Sequential()

        model.add(layers.Dense(self.layers[0], input_shape=(2,)))

        for i in range(1, len(self.layers)):
            model.add(layers.Dense(self.layers[i], activation='tanh'))

        model.add(layers.Dense(self.layers[-1]))

        return model

    def net_uv(self, x, t):
        """Get u, v and their spatial derivatives."""
        X = tf.Variable(x, dtype=tf.float32)
        T = tf.Variable(t, dtype=tf.float32)

        with tf.GradientTape() as tape:
            tape.watch([X, T])
            uv = self.build_nn(X)
            u = uv[:, 0:1]
            v = uv[:, 1:2]

        # spatial derivatives
        u_x = tape.gradient(u, X)
        v_x = tape.gradient(v, X)

        # # temporal derivatives
        # u_t = tape.gradient(u, T)
        # v_t = tape.gradient(v, T)

        del tape

        return u, v, u_x, v_x

    def net_f_uv(self, x, t):
        """Compute PDE residuals."""
        u, v, u_x, v_x = self.net_uv(x, t)

        u_t = tape.gradient(u, T)
        v_t = tape.gradient(v, T)

        u_xx = tape.gradient(u_x, X)
        v_xx = tape.gradient(v_x, X)

        f_u = u_t + 0.5*v_xx + (u**2 + v**2)*v
        f_v = v_t - 0.5*u_xx - (u**2 + v**2)*u

        return f_u, f_v
    
    def initialize_NN(self, layers):
        """
        Initialize the weights and biases for the neural network.
        
        Args:
            layers (list): List of layer sizes for the neural network
        """
        weights = []
        biases = []
        num_layers = len(layers)

        for i in range(num_layers - 1):
            W = self.xavier_init(layers[i], layers[i+1])
            b = tf.Variable(tf.zeros([1, layers[i+1]], dtype=tf.float32))
            weights.append(W)
            biases.append(b)

        return weights, biases

    def xavier_init(self, size_in, size_out):
        """
        Initialize the weights using Xavier initialization.

        Args:
            size_in (int): Input size
            size_out (int): Output size
        """
        stddev = np.sqrt(2 / (size_in + size_out))
        return tf.Variable(tf.random.truncated_normal([size_in, size_out], stddev=stddev))
    
    def compute_loss(self):
        """Compute the loss function."""
        
        